In [1]:
import cPickle as pickle
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.ensemble import RandomForestClassifier

import numpy as np
import pylab as pl
import matplotlib.pyplot as plt
import xml.etree.cElementTree as ET
from sklearn.decomposition import PCA
from sklearn.preprocessing import MaxAbsScaler
import pandas as pd
%matplotlib inline
%pylab inline

/users/rakesh.jasti/common_libs/miniconda2/envs/py27/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


In [2]:
train_p = 'data/train.pkl'
val_p = 'data/val.pkl'

with open(train_p, 'rb') as pkl:
    train_data = pickle.load(pkl)

with open(val_p, 'rb') as pkl:
    test_data = pickle.load(pkl)

In [3]:
X_train=train_data[:,:-1]
y_train=train_data[:,-1]

X_test=test_data[:,:-1]
y_test=test_data[:,-1]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=66)

In [4]:
# print sum(y_train==1)
# print sum(y_train==0)

# print sum(y_test==1)
# print sum(y_test==0)

### RandomForest

In [5]:
bdt_real = RandomForestClassifier(n_estimators=4,
    min_samples_split=3,
    random_state=66)

bdt_real.fit(X_train, y_train);

In [6]:
predicted=bdt_real.predict(X_test)
print 'Accuracy: {:.2f}'.format(accuracy_score(y_test, predicted)*100)
cnf_matrix = confusion_matrix(y_test, predicted)
print cnf_matrix

Accuracy: 51.39
[[1002  568]
 [1043  701]]


### Decision Tree

In [7]:
bdt_real = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=4),
    n_estimators=10,
    learning_rate=1, random_state=66)

bdt_real.fit(X_train, y_train);

In [8]:
predicted=bdt_real.predict(X_test)
print 'Accuracy: {:.2f}'.format(accuracy_score(y_test, predicted)*100)
cnf_matrix = confusion_matrix(y_test, predicted)
print cnf_matrix

Accuracy: 53.23
[[ 366 1204]
 [ 346 1398]]


### SVM

In [9]:
C_range = 10. ** np.arange(2, 7, 2)
gamma_range = 10. ** np.arange(-5, 5, 2)
rbf_param_grid = dict(gamma=gamma_range, C=C_range)

In [10]:
kernel = 'rbf'

if kernel=='rbf':
    grid = GridSearchCV(SVC(), param_grid=rbf_param_grid, cv=3)#StratifiedKFold(y=y_train, k=3))
elif kernel=='poly':
    grid = GridSearchCV(SVC(kernel='poly'), param_grid=poly_param_grid, cv=3)#StratifiedKFold(y=y_train, k=3))
grid.fit(X_train, y_train)
print("The best classifier is: ", grid.best_estimator_)

('The best classifier is: ', SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.10000000000000001,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))


In [11]:
best_model = grid.best_estimator_
predicted=best_model.predict(X_test)
precision, recall, fbeta_score, support = precision_recall_fscore_support(y_test, predicted,
                                                                          average='macro')
cnf_matrix = confusion_matrix(y_test, predicted)
# print "Accuracy on test data: {0:.0f}".format(score*100)
print 'fscore: {:.2f}\nPrecision: {:.2f}\nRecall: {:.2f}'.format(fbeta_score*100,precision*100,recall*100)
print cnf_matrix
print cnf_matrix/cnf_matrix.astype(np.float).sum(axis=1)
print 'Accuracy: {:.2f}'.format(accuracy_score(y_test, predicted)*100)

fscore: 34.48
Precision: 26.31
Recall: 50.00
[[   0 1570]
 [   0 1744]]
[[ 0.          0.90022936]
 [ 0.          1.        ]]
Accuracy: 52.63


/users/rakesh.jasti/common_libs/miniconda2/envs/py27/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
